# Assigment 3

In [1]:
import igl
import numpy as np
import meshplot as mp

In [2]:
v, f = igl.read_triangle_mesh("data/bunny.off")
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

# Vertex normal

### Standard face normal

In [3]:
def standard_face_normals(v, f):
    vert_norm = {}

    for i in range(v.shape[0]):
        vert_norm[i] = np.array([0.,0.,0.])

    for face in f:
        e1 = v[face[1]] - v[face[0]]
        e2 = v[face[2]] - v[face[0]]
        cross = np.cross(e1, e2)
        n = cross / np.linalg.norm(cross)

        for vert in face:
            vert_norm[vert] += n

    vn = []

    for i in range(v.shape[0]):
        vn.append(vert_norm[i] / np.linalg.norm(vert_norm[i]))

    vn = np.array(vn)
    
    return vn
    
p = mp.plot(v, f, n = standard_face_normals(v, f), shading={'flat':False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### Area-weighted face normal

In [4]:
def area_weighted_face_normal(v, f):
    d_area = igl.doublearea(v, f)

    vert_area_norm = {}

    for i in range(v.shape[0]):
        vert_area_norm[i] = np.array([0.,0.,0.])

    for i in range(f.shape[0]):
        e1 = v[f[i][1]] - v[f[i][0]]
        e2 = v[f[i][2]] - v[f[i][0]]
        n = np.cross(e1, e2)

        for vert in f[i]:
            vert_area_norm[vert] += d_area[i] * n

    vert_area_n = []

    for i in range(v.shape[0]):
        vert_area_n.append(vert_area_norm[i] / np.linalg.norm(vert_area_norm[i]))

    vert_area_n = np.array(vert_area_n)
    
    return vert_area_n

p = mp.plot(v, f, n = area_weighted_face_normal(v, f), shading={'flat':False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### Mean-curvature normal

In [5]:
def mean_curvature_normals(v, f):
    Lw = igl.cotmatrix(v, f)
    H = Lw / 2.0

    MCN = H @ v
    mcn = np.zeros(v.shape)
    vert_area_n = area_weighted_face_normal(v, f)
    vn = standard_face_normals(v, f)

    eps = 1e-4

    for i in range(MCN.shape[0]):
        if np.linalg.norm(MCN[i]) > eps:
            n = MCN[i] / np.linalg.norm(MCN[i])
            if np.dot(n, vn[i]) < 0:
                n *= -1
            mcn[i] = n
        else:
            mcn[i] = vert_area_n[i]
            
    return mcn

p = mp.plot(v, f, n=mean_curvature_normals(v, f), shading={'flat':False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### PCA normal

In [6]:
def PCA_normals(v, f, k):
    normals = np.zeros(v.shape)
    vert_norm = standard_face_normals(v, f)

    for i in range(v.shape[0]):
        part = np.argpartition(np.linalg.norm(np.abs(v - v[i]), axis=1), k)[:k]
        centroid = np.mean(v[part], axis=0)

        Y = v[part] - centroid
        S = np.dot(Y.T, Y)

        eigenvalues, eigenvectors = np.linalg.eig(S)
        vec = eigenvectors[:, np.argmin(eigenvalues)]

        vec /= np.linalg.norm(vec)
        if np.dot(vec, vert_norm[i]) < 0:
            vec = -vec
        normals[i] = vec

    return normals

p = mp.plot(v, f, n=PCA_normals(v, f, 30), shading={'flat':False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### Quadratic fitting normal

In [21]:
def project(vi, vp, norm):
    return vp - (norm * ((vp - vi).dot(norm)))

def create_ref_frame(vi, adj, height_axis):
    longest_v = project(vi, adj, height_axis) - (vi / np.linalg.norm(vi))
    longest_v /= np.linalg.norm(longest_v)
    
    y_axis = np.cross(np.array(height_axis), np.array(longest_v))
    y_axis /= np.linalg.norm(y_axis)
    
    ref = [longest_v, y_axis, height_axis]
    return ref

def quad_fit(points):
    A =  np.zeros((len(points), 6))
    b = np.zeros((len(points), 1))
    
    for i in range(len(points)):
        u = points[i][0]
        v = points[i][1]
        n = points[i][2]
        
        A[i, 0] = u * u
        A[i, 1] = u * v
        A[i, 2] = v * v
        A[i, 3] = u
        A[i, 4] = v
        A[i, 5] = 1
        
        b[i] = n
    
    lst_sq, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
    return lst_sq


def get_normal(vi, coef):
    u = vi[0]
    v = vi[1]
        
    a = coef.item(0)
    b = coef.item(1)
    c = coef.item(2)
    d = coef.item(3)
    e = coef.item(4)
    
    du = 2 * a * u + b * v + d
    dv = b * u + 2 * c * v + e
    
    Tu = np.array([1, 0, du])
    Tv = np.array([0, 1, dv])
    
    n = np.cross(Tu, Tv)
    n = n / np.linalg.norm(n)
    return n
    

def fit_quad(vi, ref, adj):
    points = v[adj]
    return quad_fit(points)

def quad_fit_norms(v, f, k):
    vert_norm = standard_face_normals(v, f)

    adj = igl.adjacency_list(f)
    norms = np.zeros(v.shape)

    for i in range(v.shape[0]):
        part = np.argpartition(np.linalg.norm(np.abs(v - v[i]), axis=1), k)[:k]
        centroid = np.mean(v[part], axis=0)

        Y = v[part] - centroid
        S = np.dot(Y.T, Y)

        eigenvalues, eigenvectors = np.linalg.eig(S)
        height_axis = eigenvectors[:, np.argmin(eigenvalues)]

        height_axis /= np.linalg.norm(height_axis)

        ref = create_ref_frame(v[i], v[adj[i][0]], height_axis)
        coef = fit_quad(v[i], ref, adj[i])
        n = get_normal(v[i], coef)


        if np.dot(n, vert_norm[i]) < 0:
            n = -n
        n /= np.linalg.norm(n)
        norms[i] = n
    return norms

p = mp.plot(v, f, n=quad_fit_norms(v, f, 30), shading={'flat':False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

## All Normals

In [8]:
p = mp.subplot(v, f, n=standard_face_normals(v, f), shading={'flat':False}, s = [3,2,0])
mp.subplot(v, f, n=area_weighted_face_normal(v, f), shading={'flat':False}, s = [3,2,1], data=p)
mp.subplot(v, f, n=mean_curvature_normals(v, f), shading={'flat':False}, s = [3,2,2], data=p)
mp.subplot(v, f, n=PCA_normals(v, f, 30), shading={'flat':False}, s = [3,2,3], data=p)
mp.subplot(v, f, n=quad_fit_norms(v, f, 30), shading={'flat':False}, s = [3,2,4], data=p)

# Curvature

In [9]:
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp

v, f = igl.read_triangle_mesh("data/bunny.off")

### Gaussian curvature

In [10]:
v1, v2, k1, k2 = igl.principal_curvature(v, f)

p = mp.plot(v, f, k1*k2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### Mean curvature

In [11]:
Lw = igl.cotmatrix(v, f)
H = sp.linalg.norm(Lw, axis=0) / 2.0
p = mp.plot(v, f, H)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### Principal curvature

In [12]:
v1, v2, k1, k2 = igl.principal_curvature(v, f)
h2 = 0.5 * (k1 + k2)

avg = igl.avg_edge_length(v, f) / 2.0

p = mp.subplot(v, f, k1, s = [2,2,0])
mp.subplot(v, f, k2, s = [2,2,1], data = p)

q = mp.plot(v, f)
q.add_lines(v + v1 * avg, v - v1 * avg, shading={'line_color':'red'})
q.add_lines(v + v2 * avg, v - v2 * avg, shading={'line_color':'green'})



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

2

# Smoothing with the Laplacian

In [13]:
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

vcow, fcow = igl.read_triangle_mesh("data/cow.off")
cow_plot = mp.plot(vcow, fcow, shading={'wireframe':True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [14]:
def identity(v):
    return sp.identity(v.shape[0])

def uniform_Laplace(v, f):
    adj = igl.adjacency_list(f)
    LU = sp.lil_matrix((v.shape[0], v.shape[0]))
    for i, x in enumerate(adj):
        LU[i,i] = -len(adj[i])
        for j in x:
            LU[i, j] = 1
    return LU

## Explicit laplacian

### Cotangent Explicit

In [15]:
def cotangent_explicit(v, f, lam, iterations):
    vnew = v.copy()
    I = identity(vnew)
    Lw = igl.cotmatrix(v, f)
    M = igl.massmatrix(vnew, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)

    for i in range(iterations):
        L = sp.diags(1/M.diagonal()) @ Lw
        vnew = (I + lam * L) @ vnew
    return vnew
    
# Produces picture from assignment with lamda = 0.01, iterations = 1000
def cotangent_explicit_original(v, f, lam, iterations):
    vnew = v.copy()
    I = identity(vnew)
    Lw = igl.cotmatrix(vnew, f)

    for i in range(iterations):
        vnew = (I + lam * Lw) @ vnew
    return vnew

p = mp.subplot(cotangent_explicit(vcow, fcow, 0.000001, 1000), fcow, shading={'wireframe':True}, s = [1,2,0])
mp.subplot(cotangent_explicit_original(vcow, fcow, 0.01, 1000), fcow, shading={'wireframe':True}, s = [2,2,1], data = p)


### Uniform Explicit

In [16]:
def uniform_explicit(v, f, lam, iterations):
    LU = uniform_Laplace(v, f)
    I = identity(v)

    vnew = v.copy()
    for i in range(iterations):
        vnew = (I + lam * LU) @ vnew

    return vnew


p = mp.plot(uniform_explicit(vcow, fcow, 0.01, 1000), fcow, shading={'wireframe':True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0011806…

## Implicit laplacian

### Uniform Implicit

In [17]:
def uniform_implicit(v, f, lam, iterations):
    I = identity(v)

    vnew = v.copy()
    LU = uniform_Laplace(vnew, f)
    A = I - LU * lam
    for i in range(iterations):
        vnew = spsolve(A, vnew)

    return vnew


p = mp.plot(uniform_explicit(vcow, fcow, 0.1, 100), fcow, shading={'wireframe':True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0011428…

### Cotangent Implicit

In [18]:
def cotangent_implicit(v, f, lam, iterations):
    vnew = v.copy()
    I = identity(vnew)
    Lw = igl.cotmatrix(vnew, f)

    for i in range(iterations):
        ''' Smoothing by flowing '''
        M = igl.massmatrix(vnew, f)
        Minv = sp.diags(1/M.diagonal())
        vnew = spsolve((I - lam * Minv @ Lw), vnew)

    return vnew

def cotangent_implicit_original(v, f, lam, iterations):
    vnew = v.copy()
    I = identity(vnew)
    Lw = igl.cotmatrix(vnew, f)
    
    for i in range(iterations):
        ''' Smoothing produces pictured version when lam = 20 '''
        vnew = spsolve((I - lam * Lw), vnew)

    return vnew

p = mp.subplot(cotangent_implicit(vcow, fcow, 0.01, 1), fcow, shading={'wireframe':True}, s = [1,2,0])
mp.subplot(cotangent_implicit_original(vcow, fcow, 20, 1), fcow, shading={'wireframe':True}, s = [1,2,1], data=p)